In [ ]:
#Ứng dụng trong các tình huống giá sản phẩm/giá vốn hàng bán thay đổi liên tục 
#Cách 1: Sử dụng Merge_asof
#Cách 2: Chia 2 trường hợp. Mỗi trường hợp xử lý trên một df rồi concat lại (Tư duy khá giống cte trong sql)
#Cách 3: Dùng Def để thiết lập hàm
File này sử dụng cách 2. 

In [56]:
import pandas as pd
prices=pd.read_csv(r"prices.csv")
sales=pd.read_csv(r"sales.csv")

In [57]:
# Join 2 bang 
df= sales.merge(prices, on="product_id", how="left")
df['ordered_at'] =  pd.to_datetime(df['ordered_at'])
df['updated_at'] =  pd.to_datetime(df['updated_at'])
print(df)


     product_id          ordered_at  quantity_ordered  old_price  new_price  \
0       3998909 2018-09-18 17:51:00                 1      15500      16500   
1       3998909 2018-09-18 17:51:00                 1      17000      15500   
2       3998909 2018-09-18 17:51:00                 1      19000      17000   
3       3998909 2018-09-18 12:52:00                 1      15500      16500   
4       3998909 2018-09-18 12:52:00                 1      17000      15500   
..          ...                 ...               ...        ...        ...   
949     4085861 2018-09-13 23:29:00                 1      67000      62500   
950          64 2018-09-11 19:32:00                 1     270000     239000   
951          64 2018-09-14 20:10:00                 1     270000     239000   
952          64 2018-09-12 16:00:00                 1     270000     239000   
953          64 2018-09-12 13:23:00                 1     270000     239000   

             updated_at  
0   2018-09-16 05:09:00  

In [58]:
#Copy data ra 2 bien df moi 
case1=df.copy()
case2=df.copy()

In [59]:
#Truong hop 1: San pham khong co updated price tai thoi diem order => lay old_price cua ngay update som nhat.
case1['min_updated']=case1.groupby(['product_id','ordered_at'])['updated_at'].transform(lambda x: x.min())
case1=case1.loc[(case1['ordered_at'] <  case1['min_updated']) & (case1['updated_at']==case1['min_updated']) ,:]
case1['price']=case1['old_price']
print(case1)

     product_id          ordered_at  quantity_ordered  old_price  new_price  \
93      4085861 2018-09-11 06:53:00                 1      60000      53500   
99      4085861 2018-09-11 08:24:00                 1      60000      53500   
105     4085861 2018-09-11 06:26:00                 1      60000      53500   

             updated_at         min_updated  price  
93  2018-09-11 08:51:00 2018-09-11 08:51:00  60000  
99  2018-09-11 08:51:00 2018-09-11 08:51:00  60000  
105 2018-09-11 08:51:00 2018-09-11 08:51:00  60000  


In [64]:
#Truong hop 2: San pham co updated price tai thoi diem order => Lay new_price cua ngay update gan nhat voi order

case2=case2.loc[(case2['ordered_at'] >  case2['updated_at'])]
case2['max_updated']=case2.groupby(['product_id','ordered_at'])['updated_at'].transform(lambda x: x.max())
case2=case2.loc[ (case2['updated_at']==case2['max_updated']) ,:]
case2['price']=case2['new_price']

print(case2)

     product_id          ordered_at  quantity_ordered  old_price  new_price  \
0       3998909 2018-09-18 17:51:00                 1      15500      16500   
3       3998909 2018-09-18 12:52:00                 1      15500      16500   
6       3998909 2018-09-18 11:33:00                 1      15500      16500   
9       3998909 2018-09-18 18:47:00                 1      15500      16500   
12      3998909 2018-09-18 17:36:00                 1      15500      16500   
..          ...                 ...               ...        ...        ...   
949     4085861 2018-09-13 23:29:00                 1      67000      62500   
950          64 2018-09-11 19:32:00                 1     270000     239000   
951          64 2018-09-14 20:10:00                 1     270000     239000   
952          64 2018-09-12 16:00:00                 1     270000     239000   
953          64 2018-09-12 13:23:00                 1     270000     239000   

             updated_at         max_updated   price

In [66]:
del case1['min_updated']
del case2['max_updated']

In [98]:
wrangled_data=pd.concat([case1,case2],axis=0)
del wrangled_data['old_price']
del wrangled_data['new_price']
wrangled_data['revenue']=wrangled_data['quantity_ordered']* wrangled_data['price']
total_revenue=wrangled_data.groupby(['product_id','price']).agg({'revenue':'sum'})

print(wrangled_data_1)
wrangled_data.to_csv(r"C:\Users\ADMIN\Downloads\Data Wrangling Practise\Data Wrangling Practise\Wrangled_data.csv")
total_revenue.to_csv(r"C:\Users\ADMIN\Downloads\Data Wrangling Practise\Data Wrangling Practise\Total_revenue.csv")

                   revenue
product_id price          
64         239000   956000
3954203    57500     57500
           60000    180000
           64000    640000
3998909    15500     15500
           16500    231000
           17000     34000
4085861    52000   1040000
           53500   2140000
           58000   2204000
           60000    180000
           62500   1812500
           67000    871000
